# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [39]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [40]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,port-gentil,-0.7193,8.7815,26.93,77,100,3.48,GA,1674362596
1,1,bilma,18.6853,12.9164,16.26,17,0,2.83,NE,1674362685
2,2,henties bay,-22.1160,14.2845,18.12,89,78,1.81,NaN,1674362686
3,3,mataura,-46.1927,168.8643,14.15,89,84,3.24,NZ,1674362686
4,4,karoi,-16.8099,29.6925,16.94,98,100,1.14,ZW,1674362686


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [41]:
%%capture --no-display

# Configure the map plot
map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City" 
)
# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [42]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_weather_df = city_data_df.loc[(city_data_df['Max Temp'] >= 23) & (city_data_df['Max Temp'] <= 30) & (city_data_df['Wind Speed'] < 3.0)]

# Drop any rows with null values
ideal_weather_df = ideal_weather_df.dropna()

# Display sample data
ideal_weather_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
9,9,mahebourg,-20.4081,57.7000,25.70,94,75,1.54,MU,1674362581
22,22,sampit,-2.5333,112.9500,28.99,70,87,1.82,ID,1674362650
50,50,sungairaya,0.7000,108.9000,28.69,71,72,2.60,ID,1674362659
52,52,faanui,-16.4833,-151.7500,27.27,65,100,2.51,PF,1674362698
56,56,puerto ayora,-0.7393,-90.3518,23.64,97,41,2.04,EC,1674362699


### Step 3: Create a new DataFrame called `hotel_df`.

In [43]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_weather_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df = hotel_df.assign(Hotel_Name = "")

# Display sample data
hotel_df.head()

,City,Country,Lat,Lng,Humidity,Hotel_Name
9,mahebourg,MU,-20.4081,57.7000,94,
22,sampit,ID,-2.5333,112.9500,70,
50,sungairaya,ID,0.7000,108.9000,71,
52,faanui,PF,-16.4833,-151.7500,65,
56,puerto ayora,EC,-0.7393,-90.3518,97,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [44]:
# Set parameters to search for a hotel
radius = 10000
limit = 20

params = {"apiKey": geoapify_key,
        "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    longitude = hotel_df.loc[index, "Lng"]
    latitude = hotel_df.loc[index, "Lat"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    params["categories"] =f"accommodation.hotel"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel_Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
            # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel_Name"] = "No hotel found"
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel_Name']}")

# Display sample data
hotel_df

Starting hotel search
mahebourg - nearest hotel: Grand Bel Air
sampit - nearest hotel: Hotel Sultan
sungairaya - nearest hotel: No hotel found
faanui - nearest hotel: Oa Oa Lodge
puerto ayora - nearest hotel: Hostal La Mirada De Solitario George
avarua - nearest hotel: Paradise Inn
pakxan - nearest hotel: Meanam hotel
bambous virieux - nearest hotel: Otentic Eco Tent Experience
victoria - nearest hotel: 四季酒店 Four Seasons Hotel
tautira - nearest hotel: No hotel found
cap malheureux - nearest hotel: Cape Garden
kandrian - nearest hotel: No hotel found
auki - nearest hotel: No hotel found
luanda - nearest hotel: Hotel Forum
mayumba - nearest hotel: Résidence Hôtelière Fabso
isangel - nearest hotel: Tanna Lodge
cayenne - nearest hotel: Central Hôtel
coxim - nearest hotel: Avendita
kavieng - nearest hotel: Kavieng Hotel
tha bo - nearest hotel: Sengthong resort
inverell - nearest hotel: Oxford Hotel
maxixe - nearest hotel: Hotel Herry
grand gaube - nearest hotel: Coco Villa
durban - nearest 

,City,Country,Lat,Lng,Humidity,Hotel_Name
9,mahebourg,MU,-20.4081,57.7000,94,Grand Bel Air
22,sampit,ID,-2.5333,112.9500,70,Hotel Sultan
50,sungairaya,ID,0.7000,108.9000,71,No hotel found
52,faanui,PF,-16.4833,-151.7500,65,Oa Oa Lodge
56,puerto ayora,EC,-0.7393,-90.3518,97,Hostal La Mirada De Solitario George
...,...,...,...,...,...,...
491,buchanan,LR,5.8808,-10.0467,86,Phillipmena's Guesthouse
505,haapiti,PF,-17.5667,-149.8667,68,No hotel found
522,floriano,BR,-6.7669,-43.0225,76,No hotel found
534,meulaboh,ID,4.1363,96.1285,65,Hotel Meuligou


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [45]:
%%capture --no-display

# Configure the map plot
map_plot = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 0.5,
    color = "City",
    hover_cols = ["Hotel_Name","Country"]
    )

# Display the map
map_plot

:Overlay
   .Tiles.I  :Tiles   [x,y]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel_Name,Country)